In [1]:
import ee
import geemap
ee.Initialize()

In [8]:
gaul_level1 = ee.FeatureCollection('FAO/GAUL/2015/level1')
countries = gaul_level1.aggregate_array('ADM0_NAME').distinct().sort()
# countries
regions = gaul_level1.aggregate_array('ADM1_NAME').distinct().sort()
# print("\nUnique Region Names (ADM1_NAME):")
# print(regions.getInfo())
# regions

In [15]:
# # GAUL Level 1 dataset
# gaul_level1 = ee.FeatureCollection('FAO/GAUL/2015/level1')
# # Get all unique country names
# countries = gaul_level1.aggregate_array('ADM0_NAME').distinct().sort()
# # print("Unique Country Names (ADM0_NAME):")
# # print(countries.getInfo())

# # Filter for Guyana only
# guyana = gaul_level1.filter(ee.Filter.eq('ADM0_NAME', 'Guyana'))

# # Get all ADM1 regions within Guyana
# guyana_regions = guyana.aggregate_array('ADM1_NAME').distinct().sort()
# print("\nUnique ADM1 Names in Guyana:")
# # print(guyana_regions.getInfo())
# guyana_regions

In [9]:
#  GAUL Level 2 dataset
gaul_level2 = ee.FeatureCollection('FAO/GAUL/2015/level2')
# Filter for Guyana
guyana_adm2 = gaul_level2.filter(ee.Filter.eq('ADM0_NAME', 'Guyana'))
# Get unique ADM2 names
guyana_cities = guyana_adm2.aggregate_array('ADM1_NAME').distinct().sort()
print("Unique ADM2 Names in Guyana (City Level):")
# print(guyana_cities.getInfo())
guyana_cities


Unique ADM2 Names in Guyana (City Level):


In [44]:
import ee

# Initialize the Earth Engine API
ee.Initialize()

gaul_level1 = ee.FeatureCollection('FAO/GAUL/2015/level1')
guyana = gaul_level1.filter(ee.Filter.eq('ADM0_NAME', 'Guyana'))
georgetown = guyana.filter(ee.Filter.eq('ADM1_NAME', 'Georgetown'))

roi = guyana.geometry()


# Define dynamic time period
start_date = "2005-01-01"
end_date = "2005-12-31"

# Load datasets
jrc = ee.ImageCollection("JRC/GSW1_4/MonthlyHistory")
srtm = ee.Image("USGS/SRTMGL1_003")

# Filter JRC dataset for the selected time period and region
jrc_filtered = jrc.filterBounds(roi).filterDate(start_date, end_date)

# Step 1: Exclude permanent water bodies
permanent_water = ee.Image("JRC/GSW1_4/GlobalSurfaceWater").select("occurrence").gte(90)
jrc_filtered = jrc_filtered.map(lambda img: img.updateMask(permanent_water.Not()))

# Step 2: Calculate Flood Frequency
def calculate_flood_frequency(collection):
    # Detect observations and water occurrences
    def add_bands(img):
        obs = img.gt(0).rename("obs")
        water = img.select("water").eq(2).rename("water")
        return img.addBands([obs, water])
    
    collection = collection.map(add_bands)
    total_obs = collection.select("obs").sum().rename("total_obs")
    total_water = collection.select("water").sum().rename("total_water")
    flood_frequency = total_water.divide(total_obs).multiply(100).rename("flood_frequency")
    return flood_frequency.updateMask(flood_frequency.neq(0))

flood_frequency = calculate_flood_frequency(jrc_filtered)

# Step 3: Analyze DEM
# Calculate slope and flat areas
slope = ee.Terrain.slope(srtm).rename("slope")
flat_area = slope.lt(5).rename("flat_area")  
low_lying = srtm.lt(10).rename("low_lying")  
# Step 4: Flood-Prone Areas
flood_prone_area = flood_frequency.updateMask(low_lying.And(flat_area))

# Step 5: Monthly Water Surface Maps
# monthly_water_surface = jrc_filtered.map(calculate_monthly_water_surface)
def get_monthly_flood_layers(collection, roi):
    """
    Extracts monthly flood layers from the given collection.
    """
    monthly_layers = {}
    months = collection.aggregate_array("system:index").getInfo()
    for month in months:
        img = collection.filter(ee.Filter.eq("system:index", month)).first()
        monthly_layers[month] = img.clip(roi).select("water")
    return monthly_layers

monthly_layers = get_monthly_flood_layers(jrc_filtered, roi)

# Visualization parameters
dem_vis = {"min": 0, "max": 50, "palette": ["ffffff", "00ff00", "007f00"]}
slope_vis = {"min": 0, "max": 30, "palette": ["ffffff", "ffcc99", "ff3300"]}
flat_area_vis = {"min": 0, "max": 1, "palette": ["ffffff", "0000ff"]}
flood_frequency_vis = {"min": 0, "max": 50, "palette": ["ffffff", "fffcb8", "0905ff"]}
flood_prone_area_vis = {"min": 0, "max": 50, "palette": ["ffffff", "ff9999", "ff0000"]}
water_surface_vis = {"min": 0, "max": 1, "palette": ["0000ff"]}

# Add layers to the map
print("Adding layers to the map...")
Map = geemap.Map()
Map.centerObject(roi, 8)
Map.addLayer(srtm.clip(roi), dem_vis, "SRTM DEM")
Map.addLayer(slope.clip(roi), slope_vis, "Slope")
Map.addLayer(flat_area.clip(roi), flat_area_vis, "Flat Areas")
Map.addLayer(flood_frequency.clip(roi), flood_frequency_vis, f"Flood Frequency from {start_date} to {end_date}")
Map.addLayer(flood_prone_area.clip(roi), flood_prone_area_vis, f"Flood-Prone Areas from {start_date} to {end_date}")

for month, layer in monthly_layers.items():
    Map.addLayer(layer, water_surface_vis, f"Water Surface {month}")

Map.addLayerControl()
Map




Adding layers to the map...


Map(center=[4.792974158655901, -58.97213873238909], controls=(WidgetControl(options=['position', 'transparent_…

In [47]:
Map.save("flood_analysis_map.html")

In [39]:
# def get_monthly_flood_layers(collection, roi):
#     """
#     Extracts monthly flood layers from the given collection.
    
#     Args:
#         collection: The filtered JRC water collection.
#         roi: The region of interest as an ee.Geometry or ee.FeatureCollection.
        
#     Returns:
#         A dictionary where keys are month indices and values are ee.Image objects.
#     """
#     monthly_layers = {}
    
#     # Get all monthly indexes
#     months = collection.aggregate_array("system:index").getInfo()
#     for month in months:
#         img = collection.filter(ee.Filter.eq("system:index", month)).first()
#         monthly_layers[month] = img.clip(roi).select("water")
    
#     return monthly_layers

In [40]:
# # Get the monthly water surface layers
# monthly_layers = get_monthly_flood_layers(jrc_filtered, roi)

# # Example: Visualize a specific month's layer on a geemap.Map
# example_month = "2005-01"  # Change this to the desired month
# example_layer = monthly_layers.get(example_month)

# if example_layer:
#     Map = geemap.Map()
#     Map.centerObject(roi, 8)
    
#     # Define visualization parameters
#     flood_vis = {"min": 0, "max": 1, "palette": ["ffffff", "0000ff"]}
    
#     # Add the example layer
#     Map.addLayer(example_layer, flood_vis, f"Flood {example_month}")
#     Map.addLayerControl()
#     Map
# else:
#     print(f"No data available for month {example_month}")


In [45]:
def display_monthly_flood_maps(collection, roi):
    monthly_map = geemap.Map()
    monthly_map.centerObject(roi, 8)
    
    # flood_vis = {"min": 0, "max": 3, "palette": ["ffffff", "ffcccc", "ff0000"]}
    # flood_vis = {"min": 0, "max": 3, "palette": ["ffffff", "0000ff"]}
    # flood_vis = {"min": 0, "max": 1, "palette": ["ffffff", "0000ff"]}
    flood_vis = {"min": 0, "max": 1, "palette": ["0000ff"]}

    # Extract and add each monthly image to the map
    months = collection.aggregate_array("system:index").getInfo()  # Get all monthly indexes
    for month in months:
        img = collection.filter(ee.Filter.eq("system:index", month)).first()
        monthly_map.addLayer(img.clip(roi).select("water"), flood_vis, f"Flood {month}")
    
    # Add layer control and return the map
    monthly_map.addLayerControl()
    return monthly_map

monthly_flood_map = display_monthly_flood_maps(jrc_filtered, roi)
monthly_flood_map

Map(center=[4.792974158655901, -58.97213873238909], controls=(WidgetControl(options=['position', 'transparent_…

In [ ]:
# # Optional: Export the flood frequency and flood risk results to Google Drive
# task_flood_frequency = ee.batch.Export.image.toDrive(
#     image=flood_frequency,
#     description="FloodFrequency",
#     folder="EarthEngineExports",
#     scale=30,
#     region=roi,
#     maxPixels=1e13
# )
# task_flood_frequency.start()

# task_flood_risk = ee.batch.Export.image.toDrive(
#     image=flood_risk,
#     description="FloodRisk",
#     folder="EarthEngineExports",
#     scale=30,
#     region=roi,
#     maxPixels=1e13
# )
# task_flood_risk.start()

# print("Export tasks started. Check your Google Drive for results.")

In [ ]:
# import ee
# import geemap

# # Initialize Earth Engine
# ee.Initialize()

# # Define ROI (Guyana)
# gaul = ee.FeatureCollection("FAO/GAUL/2015/level0")
# guyana = gaul.filter(ee.Filter.eq("ADM0_NAME", "Guyana"))
# roi = guyana.geometry()

# # Load datasets
# srtm = ee.Image("USGS/SRTMGL1_003")
# jrc = ee.ImageCollection("JRC/GSW1_4/MonthlyHistory")

# # Define time period for flood analysis
# start_date = "2005-01-01"
# end_date = "2005-12-31"

# # Filter JRC dataset for the time period and region
# jrc_filtered = jrc.filterBounds(roi).filterDate(start_date, end_date)

# # Exclude permanent water bodies
# permanent_water = ee.Image("JRC/GSW1_4/GlobalSurfaceWater").select("occurrence").gte(90)
# jrc_filtered = jrc_filtered.map(lambda img: img.updateMask(permanent_water.Not()))

# # Step 1: Calculate Flood Frequency
# def calculate_flood_frequency(collection):
#     def add_bands(img):
#         obs = img.gt(0).rename("obs")
#         water = img.select("water").eq(2).rename("water")
#         return img.addBands([obs, water])
    
#     collection = collection.map(add_bands)
#     total_obs = collection.select("obs").sum().rename("total_obs")
#     total_water = collection.select("water").sum().rename("total_water")
#     flood_frequency = total_water.divide(total_obs).multiply(100).rename("flood_frequency")
#     return flood_frequency.updateMask(flood_frequency.neq(0))

# flood_frequency = calculate_flood_frequency(jrc_filtered)

# # Step 2: Generate Watersheds
# # # Calculate flow direction and accumulation
# # flow_dir = ee.Terrain.flowDirection(srtm)
# # flow_accum = ee.Terrain.flowAccumulation(srtm)

# # # Define threshold for streams
# # stream_threshold = 1000
# # streams = flow_accum.gte(stream_threshold)

# # # Identify pour points (outlets)
# # pour_points = streams.updateMask(streams).reduceToVectors(
# #     geometryType="point",
# #     reducer=ee.Reducer.countEvery(),
# #     scale=30,
# #     maxPixels=1e9,
# # )

# # # Compute watersheds
# # watersheds = flow_dir.catchment(pour_points, scale=30)

# # Step 3: Analyze DEM
# # Calculate slope and low-lying areas
# slope = ee.Terrain.slope(srtm).rename("slope")
# flat_area = slope.lt(5).rename("flat_area")  # Slope < 5 degrees
# low_lying = srtm.lt(20).rename("low_lying")  # Elevation < 20 meters

# # Step 4: Flood-Prone Areas
# flood_prone_area = flood_frequency.updateMask(low_lying.And(flat_area))

# # Visualization parameters
# dem_vis = {"min": 0, "max": 50, "palette": ["ffffff", "00ff00", "007f00"]}
# slope_vis = {"min": 0, "max": 30, "palette": ["ffffff", "ffcc99", "ff3300"]}
# flat_area_vis = {"min": 0, "max": 1, "palette": ["ffffff", "0000ff"]}
# flood_frequency_vis = {"min": 0, "max": 50, "palette": ["ffffff", "fffcb8", "0905ff"]}
# flood_prone_area_vis = {"min": 0, "max": 50, "palette": ["ffffff", "ff9999", "ff0000"]}
# watershed_vis = {"color": "blue", "opacity": 0.5}
# stream_vis = {"min": 0, "max": 1, "palette": ["0000ff"]}

# # Visualization in Map
# Map = geemap.Map()
# Map.centerObject(roi, 8)

# # Add layers
# Map.addLayer(srtm.clip(roi), dem_vis, "SRTM DEM")
# Map.addLayer(slope.clip(roi), slope_vis, "Slope")
# Map.addLayer(flat_area.clip(roi), flat_area_vis, "Flat Areas")
# Map.addLayer(flood_frequency.clip(roi), flood_frequency_vis, "Flood Frequency")
# Map.addLayer(flood_prone_area.clip(roi), flood_prone_area_vis, "Flood-Prone Areas")
# # Map.addLayer(watersheds, watershed_vis, "Watersheds")
# # Map.addLayer(streams.clip(roi), stream_vis, "Streams")

# # Display the Map
# Map.addLayerControl()
# Map
